In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = ""

In [12]:
!pip install -q langchain_google_genai langchain-core requests google-ai-generativelanguage

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [14]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [15]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [16]:
multiply.name

'multiply'

In [17]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [18]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [19]:
# tool binding

In [20]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2)

In [21]:
llm.invoke('hi')

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--51aec5b7-5548-462b-9c00-7ac6253a825e-0', usage_metadata={'input_tokens': 2, 'output_tokens': 371, 'total_tokens': 373, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 361}})

In [22]:
llm_with_tools = llm.bind_tools([multiply])

In [23]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm doing well, thank you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--c650cd8d-c41d-4712-8eeb-a79e9778a97a-0', usage_metadata={'input_tokens': 58, 'output_tokens': 16, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}})

In [24]:
query = HumanMessage('can you multiply 3 with 1000')

In [25]:
messages = [query]

In [26]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [27]:
result = llm_with_tools.invoke(messages)

In [28]:
messages.append(result)

In [29]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3, "b": 1000}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--432b0e40-0d6a-4085-a99e-f46b0c2c8824-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': '5cab4fd0-9aa7-4eea-86ed-1a58be4b5244', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 97, 'total_tokens': 162, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 76}})]

In [30]:
tool_result = multiply.invoke(result.tool_calls[0])

In [31]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='5cab4fd0-9aa7-4eea-86ed-1a58be4b5244')

In [32]:
messages.append(tool_result)

In [33]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3, "b": 1000}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--432b0e40-0d6a-4085-a99e-f46b0c2c8824-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': '5cab4fd0-9aa7-4eea-86ed-1a58be4b5244', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 97, 'total_tokens': 162, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 76}}),
 ToolMessage(content='3000', name='multiply', tool_call_id='5cab4fd0-9aa7-4eea-86ed-1a58be4b5244')]

In [34]:
llm_with_tools.invoke(messages).content

'The product of 3 and 1000 is 3000.'

In [35]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [36]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [37]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1760486401,
 'time_last_update_utc': 'Wed, 15 Oct 2025 00:00:01 +0000',
 'time_next_update_unix': 1760572801,
 'time_next_update_utc': 'Thu, 16 Oct 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.8152}

In [38]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [40]:
# tool binding
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash',temperature=0)

In [41]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [42]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [43]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [44]:
ai_message = llm_with_tools.invoke(messages)

In [45]:
messages.append(ai_message)

In [46]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'USD', 'base_currency': 'INR'},
  'id': 'ecf52c40-27b3-418d-94cb-75e98b2cb63e',
  'type': 'tool_call'}]

In [47]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [48]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--30e88c3f-9937-41f1-96d2-71baed550586-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'INR'}, 'id': 'ecf52c40-27b3-418d-94cb-75e98b2cb63e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 92, 'output_tokens': 13, 'total_tokens': 105, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms",

In [ ]:
llm_with_tools.invoke(messages).content

In [49]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

/tmp/ipython-input-627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [50]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "I am doing well, thank you for asking. How can I help you today?"
}
```

> Finished chain.
